In [1]:
import ase, pyscf, os
from pyscf import gto

As an example, I will be creating a trajectory out of the various datasets contained within the [GMTKN55](https://github.com/grimme-lab/GMTKN55) database. I have cloned this repository and will be iterating through the folders and their contents to create the objects needed.

In [2]:
repo_dir = '/home/awills/Documents/Research/GMTKN55'
subdirs = [i for i in os.listdir(repo_dir) if os.path.isdir(os.path.join(repo_dir, i)) and '.' not in i and 'utils' not in i]

In [3]:
subdirs

['PA26',
 'CHB6',
 'G21IP',
 'ISO34',
 'YBDE18',
 'UPU23',
 'MCONF',
 'INV24',
 'BH76',
 'CDIE20',
 'ICONF',
 'BHROT27',
 'CARBHB12',
 'RSE43',
 'BUT14DIOL',
 'C60ISO',
 'DC13',
 'NBPRC',
 'ALKBDE10',
 'BHPERI',
 'IL16',
 'DARC',
 'BSR36',
 'SCONF',
 'WATER27',
 'DIPCS10',
 'Amino20x4',
 'PCONF21',
 'S22',
 'G2RC',
 'ACONF',
 'HEAVYSB11',
 'S66',
 'ADIM6',
 'TAUT15',
 'PArel',
 'ISOL24',
 'HAL59',
 'W4-11',
 'WCPT18',
 'PX13',
 'IDISP',
 'MB16-43',
 'G21EA',
 'AHB21',
 'PNICO23',
 'AL2X6',
 'RC21',
 'ALK8',
 'FH51',
 'HEAVY28',
 'SIE4x4',
 'RG18',
 'BHDIV10']

In [4]:
sets = {dirname : {} for dirname in subdirs}
for subdir in subdirs:
    lookdir = os.path.join(repo_dir, subdir)
    subsubdirs = [i for i in os.listdir(lookdir) if os.path.isdir(os.path.join(lookdir, i))]
    sets[subdir] = {i: [] for i in subsubdirs}

In [5]:
sets

{'PA26': {'Cp': [],
  'phosphapyrrolp': [],
  'C': [],
  'ch2s': [],
  'ch2sp': [],
  'c2h2': [],
  'sih4p': [],
  'ch3cooh': [],
  'p4p': [],
  'p2p': [],
  'c2f6p': [],
  'Gp': [],
  'T': [],
  'nh3p': [],
  'c2f6': [],
  'ch3coohp': [],
  'gly': [],
  'si2h6': [],
  'sih4': [],
  'phenol': [],
  'c2h2p': [],
  'p2': [],
  'ass': [],
  'h2p': [],
  'Tp': [],
  'h2s': [],
  'G': [],
  'p4': [],
  'cysp': [],
  'cys': [],
  'hcl': [],
  'h2': [],
  'h2sp': [],
  'h2op': [],
  'phosphapyrrol': [],
  'p6': [],
  'h2o': [],
  'si2h6p': [],
  'Ap': [],
  'ph3': [],
  'nh3': [],
  'phenolp': [],
  'ethanol': [],
  'p8': [],
  'glyp': [],
  'p6p': [],
  'A': [],
  'hclp': [],
  'ethanolp': [],
  'p8p': [],
  'assp': [],
  'ph3p': []},
 'CHB6': {'22': [],
  '25B': [],
  '23B': [],
  '27A': [],
  '22A': [],
  '24B': [],
  '26': [],
  '27': [],
  '23': [],
  '27B': [],
  '24': [],
  '22B': [],
  '25': [],
  '26A': [],
  '26B': [],
  '24A': [],
  '25A': [],
  '23A': []},
 'G21IP': {'al': [],
  '

For each of these configurations, then, we will read in the `struc.xyz` file for atomic coordinates, while searching in the `coord` files for any specifications like `$eht charge=X unpaired=Y` to give us the needed charges and $S$ values to feed PySCF (which, luckily, is not the true physical spin, just the number of unpaired electrons per [this page](https://pyscf.org/user/gto.html#spin-and-charge)).

In the `coord` files, the only extra information is the charge/spin, flagged by the `$eht` tag before the charge and unpaired electrons.

We will parse the `.xyz` files for the atoms and coordinates, given the fixed file format, and then search the `coord` for the extra specifications, and if not present we assume that both are 0.

The `PBEh-3c_reactions.csv` file in the base `repo_dir` contains the various stoichiometric coefficients and final references to use.

In [6]:
for k, v in sets.items():
    #k is the subset name, v is the dictionary with the subset's configurations
    for ck, cv in v.items():
        print(k, ck)
        #ck is the configuration 'name', cv will end up being the molecule we create for use in pyscf calculations
        struc = os.path.join(repo_dir, k, ck, 'struc.xyz')
        coord = os.path.join(repo_dir, k, ck, 'coord')
        print("struc.xyz file: ", struc)
        print("coord file: ", coord)
        #we read through the xyz file now; the first line will be the number of atoms, and the second line can be skipped.
        with open(struc, 'r') as sf:
            #first read number of atoms, for later verification
            nat = int(sf.readline())
            _ = sf.readline()
            arr = []
            for i in range(nat):
                split_line = sf.readline().split()
                arr.append([split_line[0]] + [float(i) for i in split_line[1:]])
            assert len(arr) == nat #the number of atoms in the array must be what we read in the .xyz file
        spin = 0
        charge = 0
        with open(coord, 'r') as cf:
            #now we see if there are any charge/spin specifications
            for line in cf:
                if ('eht' in line) or ('charge' in line) or ('unpaired' in line):
                    print("EXTRA INFORMATION FOUND")
                    #there should always be whitespace between eht and charge=X, and between charge=X and unpaired=Y
                    search = line.split()
                    print(search)
                    for s in search:
                        if 'charge' in s:
                            print("CHARGE VALUE FOUND")
                            new = s.split('=')
                            charge = int(new[-1])
                        if 'unpaired' in s:
                            print("SPIN VALUE FOUND")
                            new = s.split('=')
                            spin = int(new[-1])
        print("Creating mol object with ", arr)
        print(f"mol charge = {charge}; mol spin = {spin}")
        this_mol = gto.Mole(atom = arr, charge = charge, spin = spin)
        print(this_mol.charge, this_mol.spin)
        #basis can be set here if desired.
        #during calculation a warning is printed saying that the mol object should be initialized first, which requires a call to this_mol.build()
        #this build will fail for some systems that have atoms with particular basis set needs, which is why I do not initialize by default
        #for instance, some configuration with a Pb atom in it failed, since there aren't basis sets for Pb in the default bases in PySCF, it seems?
        #I instead let the PySCF driver do that later on
        #this_mol.basis = '.....'
        #this_mol.build()
        sets[k][ck] = this_mol
        print(10*'=')
       # print(nat, arr)

PA26 Cp
struc.xyz file:  /home/awills/Documents/Research/GMTKN55/PA26/Cp/struc.xyz
coord file:  /home/awills/Documents/Research/GMTKN55/PA26/Cp/coord
EXTRA INFORMATION FOUND
['$eht', 'charge=1', 'unpaired=0']
CHARGE VALUE FOUND
SPIN VALUE FOUND
Creating mol object with  [['O', 2.58304009152147, 1.24322978382011, 0.0], ['N', 0.31932007754232, 1.09472616904147, 0.0], ['N', -1.97481255463986, 1.02155426040349, 0.0], ['N', 1.61176432673783, -0.813294304454, 0.0], ['C', -0.83089338753719, 0.38835190985829, 0.0], ['C', -0.73854057329179, -1.02553902837552, 0.0], ['C', 1.59400181326617, 0.55637239788718, 0.0], ['C', 0.49608125821435, -1.57163935274193, 0.0], ['H', -1.62589705267502, -1.63977068586491, 0.0], ['H', 0.65107145560377, -2.64119284578164, 0.0], ['H', -2.84351114271472, 0.50860886366643, 0.0], ['H', -2.04293619377059, 2.02903205681331, 0.0], ['H', 2.52036285543378, -1.25950967390478, 0.0], ['H', 0.2809490263095, 2.10907044963251, 0.0]]
mol charge = 1; mol spin = 0
1 0
PA26 phosphapy

In [7]:
for k, v in sets.items():
    for vk, vv in v.items():
        print(k, vk, vv.charge, vv.spin)

PA26 Cp 1 0
PA26 phosphapyrrolp 1 0
PA26 C 0 0
PA26 ch2s 0 0
PA26 ch2sp 1 0
PA26 c2h2 0 0
PA26 sih4p 1 0
PA26 ch3cooh 0 0
PA26 p4p 1 0
PA26 p2p 1 0
PA26 c2f6p 1 0
PA26 Gp 1 0
PA26 T 0 0
PA26 nh3p 1 0
PA26 c2f6 0 0
PA26 ch3coohp 1 0
PA26 gly 0 0
PA26 si2h6 0 0
PA26 sih4 0 0
PA26 phenol 0 0
PA26 c2h2p 1 0
PA26 p2 0 0
PA26 ass 0 0
PA26 h2p 1 0
PA26 Tp 1 0
PA26 h2s 0 0
PA26 G 0 0
PA26 p4 0 0
PA26 cysp 1 0
PA26 cys 0 0
PA26 hcl 0 0
PA26 h2 0 0
PA26 h2sp 1 0
PA26 h2op 1 0
PA26 phosphapyrrol 0 0
PA26 p6 0 0
PA26 h2o 0 0
PA26 si2h6p 1 0
PA26 Ap 1 0
PA26 ph3 0 0
PA26 nh3 0 0
PA26 phenolp 1 0
PA26 ethanol 0 0
PA26 p8 0 0
PA26 glyp 1 0
PA26 p6p 1 0
PA26 A 0 0
PA26 hclp 1 0
PA26 ethanolp 1 0
PA26 p8p 1 0
PA26 assp 1 0
PA26 ph3p 1 0
CHB6 22 1 0
CHB6 25B 0 0
CHB6 23B 0 0
CHB6 27A 1 0
CHB6 22A 1 0
CHB6 24B 0 0
CHB6 26 1 0
CHB6 27 1 0
CHB6 23 1 0
CHB6 27B 0 0
CHB6 24 1 0
CHB6 22B 0 0
CHB6 25 1 0
CHB6 26A 1 0
CHB6 26B 0 0
CHB6 24A 1 0
CHB6 25A 1 0
CHB6 23A 1 0
G21IP al 0 1
G21IP 44 0 2
G21IP na 0 1
G21

In [8]:
len(sets['PA26']['Cp'].atom)

14

With the PySCF `mol` objects created, you can translate the information into an `ase.Atoms` object as desired, or keep the dictionaries in terms of the PySCF objects.

Now we want to do CCSD(T) calculations on these objects.

In [9]:
from pyscf import scf, cc

In [10]:
calc_ens = {subdir: {k: 0 for k in sets[subdir].keys()} for subdir in subdirs}

In [11]:
calc_ens

{'PA26': {'Cp': 0,
  'phosphapyrrolp': 0,
  'C': 0,
  'ch2s': 0,
  'ch2sp': 0,
  'c2h2': 0,
  'sih4p': 0,
  'ch3cooh': 0,
  'p4p': 0,
  'p2p': 0,
  'c2f6p': 0,
  'Gp': 0,
  'T': 0,
  'nh3p': 0,
  'c2f6': 0,
  'ch3coohp': 0,
  'gly': 0,
  'si2h6': 0,
  'sih4': 0,
  'phenol': 0,
  'c2h2p': 0,
  'p2': 0,
  'ass': 0,
  'h2p': 0,
  'Tp': 0,
  'h2s': 0,
  'G': 0,
  'p4': 0,
  'cysp': 0,
  'cys': 0,
  'hcl': 0,
  'h2': 0,
  'h2sp': 0,
  'h2op': 0,
  'phosphapyrrol': 0,
  'p6': 0,
  'h2o': 0,
  'si2h6p': 0,
  'Ap': 0,
  'ph3': 0,
  'nh3': 0,
  'phenolp': 0,
  'ethanol': 0,
  'p8': 0,
  'glyp': 0,
  'p6p': 0,
  'A': 0,
  'hclp': 0,
  'ethanolp': 0,
  'p8p': 0,
  'assp': 0,
  'ph3p': 0},
 'CHB6': {'22': 0,
  '25B': 0,
  '23B': 0,
  '27A': 0,
  '22A': 0,
  '24B': 0,
  '26': 0,
  '27': 0,
  '23': 0,
  '27B': 0,
  '24': 0,
  '22B': 0,
  '25': 0,
  '26A': 0,
  '26B': 0,
  '24A': 0,
  '25A': 0,
  '23A': 0},
 'G21IP': {'al': 0,
  '44': 0,
  'na': 0,
  'IP_73': 0,
  'h': 0,
  'c': 0,
  'IP_71': 0,
  's

In [12]:
#only consider configurations with size of at most 4 atoms
MAX_ATOMS = 4
for subdir in subdirs:
    for name,config in sets[subdir].items():
        if len(config.atom) > MAX_ATOMS:
            continue
        print(name, config.atom)
        print(f"charge = {config.charge}; spin = {config.spin}")
        #basis can be set here if desired and not set above.
        #a warning is printed saying that the mol object should be initialized first, which requires a call to config.build()
        #this build will fail for some systems that have atoms with particular basis set needs, which is why I do not initialize by default
        #and instead let the PySCF driver do that.
        #eg:
        #config.basis = '....'
        #config.build()
        print('Calculating HF energy...')
        mf = scf.HF(config).run()
        print('Calculating CCSD energy...')
        this_cc = cc.CCSD(mf).run()
        print('Calculating CCSD(T) energy correction...')
        triples = this_cc.ccsd_t()
        calc_ens[subdir][name] = [mf.e_tot, this_cc.e_tot, triples, this_cc.e_tot + triples]
        print(subdir, name, this_cc.e_tot + triples)
        print(10*'=')
        

ch2s [['S', 0.0, 0.0, 1.4943317917249], ['C', 0.0, 0.0, -0.11006129644405], ['H', -0.91848640419428, 0.0, -0.69213524764042], ['H', 0.91848640419428, 0.0, -0.69213524764042]]
charge = 0; spin = 0
Calculating HF energy...


Initialize <pyscf.gto.mole.Mole object at 0x7f145a6c79e0> in <pyscf.scf.hf.RHF object at 0x7f1456da67b0>


converged SCF energy = -431.670046940286
Calculating CCSD energy...
E(CCSD) = -431.802197110042  E_corr = -0.132150169755571
Calculating CCSD(T) energy correction...
CCSD(T) correction = -0.000983345911054373
PA26 ch2s -431.803180455953
c2h2 [['C', 0.0, 0.0, 0.59872436907236], ['H', 0.0, 0.0, 1.66174695757252], ['C', 0.0, 0.0, -0.59872436907236], ['H', 0.0, 0.0, -1.66174695757252]]
charge = 0; spin = 0
Calculating HF energy...
converged SCF energy = -75.8538656931958
Calculating CCSD energy...


Initialize <pyscf.gto.mole.Mole object at 0x7f1457866c00> in <pyscf.scf.hf.RHF object at 0x7f14dc54cce0>


E(CCSD) = -76.02221195229262  E_corr = -0.1683462590968597
Calculating CCSD(T) energy correction...
CCSD(T) correction = -0.000639170949496678
PA26 c2h2 -76.02285112324212
h2p [['H', -0.25413581816518, -0.44017614908518, 0.0], ['H', -0.25413581816518, 0.44017614908518, 0.0], ['H', 0.50827163633036, 0.0, 0.0]]
charge = 1; spin = 0
Calculating HF energy...
converged SCF energy = -1.23902269996405
Calculating CCSD energy...
E(CCSD) = -1.263718138908693  E_corr = -0.02469543894464368
Calculating CCSD(T) energy correction...
CCSD(T) correction = -4.82770216073718e-19
PA26 h2p -1.2637181389086933
h2s [['S', 0.0, 0.0, -0.61815041434512], ['H', -0.97553229675618, 0.0, 0.30907520717256], ['H', 0.97553229675618, 0.0, 0.30907520717256]]
charge = 0; spin = 0
Calculating HF energy...


...

Initialize <pyscf.gto.mole.Mole object at 0x7f14578812b0> in <pyscf.scf.uhf.UHF object at 0x7f14580f5e50>


E(UCCSD) = -786.3911262178448  E_corr = -0.07239314083856985
Calculating CCSD(T) energy correction...
UCCSD(T) correction = -0.000609775524065589
G21IP 44 -786.3917359933688
na [['Na', 0.0, 0.0, 0.0]]
charge = 0; spin = 1
Calculating HF energy...


Initialize <pyscf.gto.mole.Mole object at 0x7f1457881220> in <pyscf.scf.uhf.UHF object at 0x7f1458023830>


converged SCF energy = -159.66842480153  <S^2> = 0.75034799  2S+1 = 2.000348
Calculating CCSD energy...



KeyboardInterrupt

